# Wildfire Season Summary

Record-breaking heat has made headlines around the world this year. Spring and summer were also warmer than normal on the BC South Coast; however, the precipitation deficit was the bigger story. Hot and dry weather led to a very long and challenging wildfire season in British Columbia. This section summarizes the local and provincial wildfire season, and the historic drought conditions experienced. 

The table below shows drought levels in the Lower Mainland basin for 2015 to 2023. Drought levels reached the highest level (level 5) on August 17th this year. In 2022, this occurred at the end of September. In level 5, adverse impacts are almost certain. The lower mainland has experienced significant drought for the past three summers.

In [1]:
#Import libraries
import os
import pandas as pd
import subprocess
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import json
from pandas import json_normalize

## Provincial Wildfire Summary

Both British Columbia and Canada experienced record-breaking and devastating wildfire seasons. As of October 31, there have been 2245 fires, and 2.85 million hectares of forest has burned this year in BC. This is double the previous record for area burned, which occurred in 2018. Nationally, over 18 million hectares of forest has burned (compared to a 10-year average of around 2.6 million hectares). Much of the area burned was in the northern boreal forests. 

Below you can see current BC Wildfire Statistics in comparison to past seasons. All fire zones in the province have seen significant fire activity this season. Source: [BCWS](https://www2.gov.bc.ca/gov/content/safety/wildfire-status/about-bcws/wildfire-statistics/wildfire-averages#:~:text=Wildfire%20Averages%20%20%20%20Year%20%20,%20375%20%2845%25%29%20%2012%20more%20rows%20).

In [4]:
fires = pd.read_csv('data/fire_stats.csv', parse_dates= ['date'])
fires['year'] = pd.DatetimeIndex(fires['date']).year

In [5]:
danger = pd.read_csv('data/seyfw_danger.csv', parse_dates= ['datetime']) 
danger['year'] = pd.DatetimeIndex(danger['datetime']).year
danger['month'] = pd.DatetimeIndex(danger['datetime']).month
danger['DOY'] = pd.DatetimeIndex(danger['datetime']).dayofyear

In [6]:
danger_filter = danger.loc[(danger['DGR']>=3)]


In [7]:
danger_count = danger_filter.groupby('year').count().reset_index()
df_dgr = danger_count.loc[:,'year':'datetime']
df_dgr = df_dgr.rename(columns={"year":"year", "datetime": "fire_danger"})
df_dgr['moving_avg'] = df_dgr['fire_danger'].rolling(10).mean()


In [8]:
#Area burned chart

bar = alt.Chart(fires).mark_bar(opacity=0.8, color='darkgray', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('area_burned:Q', title='Area Burned in Hectares', axis=alt.Axis(format='s')),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('area_burned', title="Area Burned")]
).properties(width=350, height=250, title='BC Area Burned per Year')


bar2 = alt.Chart(fires).mark_bar(opacity=0.8, color='red', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('fire_no:Q', title='Number of Fires', axis=alt.Axis(format='s')),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_no', title="Number of Fires")]
).properties(width=350, height=250, title='BC Number of Fires')


bar | bar2


alt.HConcatChart(...)

### Watershed Wildfire Conditions
Watershed forest fire danger rating is now rated **VERY LOW or LOW** as of September 30, 2023. Recent rainfall has lowered fire danger and significantly reduced the risk of fire starts on the BC South Coast. It's likely the forest fire season is now over for this year. 

The plot below shows the number of days where fire danger was rated moderate (3), high (4) or extreme (5) in the Seymour Watershed. This varies from season to season, but there has been an increasing trend over time, especially for high and extreme ratings. This year saw the most days in high and extreme fire danger, and the second most in moderate or higher (109 total days). 

In [149]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days rated moderate (3), high (4), and extreme (5) fire danger",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')


domain = [3, 4, 5]
range_ = ['orange', '#FE3F46', 'darkred']

test2 = alt.Chart(danger_filter, title=title).mark_bar(opacity=0.7).encode(
    alt.X('year:O', title=None),
    alt.Y('count(DGR):Q', title = 'Total days'),
    alt.Color("DGR:O", title='Danger Rating', scale=alt.Scale(domain=domain, range=range_)),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('count(DGR)', title="Number of Days")]
)

rule2 = alt.Chart(df_dgr).mark_line(color='black', opacity=0.7, strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Total days')
)

(test2 + rule2).properties(width=600)

alt.LayerChart(...)

In [150]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in High or Extreme Fire Danger",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar1 = alt.Chart(df_dgr, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('fire_danger:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_danger', title="Number of Days")]
)

rule1 = alt.Chart(df_dgr).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)


In [151]:
# Import data for crossover conditions and format dates
df = pd.read_csv('data/seyfw_crossover.csv', parse_dates= ['datetime']) 
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['month'] = pd.DatetimeIndex(df['datetime']).month
df['DOY'] = pd.DatetimeIndex(df['datetime']).dayofyear

In [152]:
# Filter dataframe for crossover days
df_filter = df.loc[(df['temp']>=30) & (df['rh'] <= 30)]

In [153]:
# Count crossover days per year
df_count = df_filter.groupby('year').count().reset_index()

In [154]:
# Filter for one column and change the name of the column
df_co = df_count.loc[:,'year':'datetime']
df_co = df_co.rename(columns={"year":"year", "datetime": "crossover_count"})
df_co['moving_avg'] = df_co['crossover_count'].rolling(10).mean()

In [155]:
# Export dataframe
# df_co.to_csv('seyfw_crossover_counts.csv', index=False)

The plot below shows the number of days where 30/30 crossover conditions were experienced each year. 30/30 crossover refers to days where temperatures exceed 30 degrees Celsius and relative humidity is lower than 30%. These are very hot and dry days! During these conditions it's likely to see explosive fire behaviour. The red dashed line shows the 10-year moving average. In the past twenty years the number of days with crossover conditions has increase from around 4 to 10. This summer we experienced 15 crossover days, which is third behind 2015 and 2018. 

In [156]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in Crossover Condition (30/30) per Year",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar = alt.Chart(df_co, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('crossover_count:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('crossover_count', title="Number of Days")]
)

rule = alt.Chart(df_co).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)

(bar + rule).properties(width=600)

alt.LayerChart(...)

### Wildfire Smoke

Smoke from wildfires in western Canada has resulted in very poor air quality in many parts of North America this summer. Early in the season, smoke from northern BC fires tended to drift east and south. This caused poor air quality in many parts of the prairies and northeastern US states. New York experienced a period of very poor air quality in early June.

Interior BC fires flared up in mid-August. As the weather pattern changed, smoke drifted south and west toward the Lower Mainland. Metro Vancouver issued air quality advisories for August 19-22, and 25-29 due to unhealthy concentration of fine particulate matter. Wildfire smoke continued to plague parts of North America well into October. Here is an article by CBC showing [just how smoky it was in BC](https://www.cbc.ca/news/canada/british-columbia/smoke-canada-british-columbia-1.7003948) in 2023. 

```{figure} img/fire_smoke.png
---
name: fire-smoke
---
Smoke impacting southern BC and the western United States (August 19, 2023)
```

Canadian wildfires have resulted in record carbon emissions in 2023. In fact, total estimated carbon emissions are over three times greater than the previous record, and six times greater than the 20-year average. 

Significant fire activity started in early May and has continued without respite for the last four months. Most provinces in Canada has dealt with challenging wildfire seasons this year. 

Data for the plot below comes from the Copernicus Atmospheric Monitoring Service (CAMS). CAMS uses satellite observations of Fire radiative power (FRP), which is a measure of fire intensity and the heat of active fires, to estimate emissions of the different pollutants that make up the smoke. 

In [15]:
emission = pd.read_csv('data/emission.csv', parse_dates=['date'])

In [17]:
title = alt.TitleParams(
   text='Estimated Fire Carbon Emissions for Canada',
   subtitle="Copernicus Atmospheric Monitoring Service",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

domain = ['2023', 'Mean (2003-2022)']
range_ = ['red', 'gray']

#soil_line.properties(width=600)

emission_line = alt.Chart(emission, title=title).mark_line().encode(
    alt.X('monthdate(date):T', title=None),
    alt.Y('carbon:Q', title='Total Carbon Emissions (megatons)', scale=alt.Scale(domain=[0,500])),
    alt.Color('year:N', title="Year", scale=alt.Scale(domain=domain, range=range_)),
    tooltip=[alt.Tooltip('year', title='Date'), alt.Tooltip('carbon', title="Total Carbon")]
)

emission_line.properties(width=600)

alt.Chart(...)